In [9]:
import re
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.spatial.distance import jaccard, cosine, hamming
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from gensim.models import Word2Vec, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
df = pd.read_csv("./data/all_titles_processed.csv", usecols=["name", "description", "responsibility", "salaryYearly", "salaryHourly", "related_titles", "skills"])
# df = df.drop_duplicates()
df.head()

FileNotFoundError: ignored

In [ ]:
df_link_skills = pd.read_csv("linked_skills_2307.txt", sep="; ", header=None, usecols=[0, 2])
df_link_skills.columns = ["crawled_skills", "api_skills"]
link_skills = df_link_skills.to_numpy()
crawl2api = {}
for e1, e2 in link_skills:
    crawl2api[e1] = e2
# crawl2api
api2crawl = defaultdict(list)
for e1, e2 in link_skills:
    api2crawl[e2].append(e1)
# api2crawl

/Users/shawn/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
api_skills = []
for skills in df.skills:
    skillset = skills.split(", ")
    curr = [crawl2api[s.lower()].title() for s in skillset]
#     curr = [re.sub(r'\([^)]*\)', '', crawl2api[s.lower()].title()).strip(" ") for s in skillset]
    api_skills.append("; ".join(sorted(set(curr))))
df["skills"] = api_skills
df.rename(columns={"related_titles": "relatedTitles"}, inplace=True)

In [ ]:
df

,name,description,responsibility,salaryYearly,salaryHourly,relatedTitles,skills
0,.NET Application Architect,A .NET developer is an information technology ...,Designing interfaces for client use\nIdentifyi...,174308.0,NaN,Enterprise Architect; Systems Architect; Infra...,Agilent Vee (Domain-Specific Programming Langu...
1,.NET Architect,Software architects are expert software design...,Perform research for projects to determine the...,170168.0,NaN,Architectural Drafter; Architectural Designer;...,Active Directory; Azure Mfa; Css Grid; Framewo...
2,.NET Developer,A .NET developer is an information technology ...,Designing interfaces for client use\nIdentifyi...,105292.0,NaN,Software Engineer; Software Developer; Front E...,.Net Framework; Agilent Vee (Domain-Specific P...
3,.NET Front End Developer,Front end developers implement the visual and ...,Designing or editing websites or web applicati...,85016.0,NaN,Software Engineer; Software Developer; Mobile ...,Agilent Vee (Domain-Specific Programming Langu...
4,.NET Full Stack Developer,A full stack developer develops and deploys th...,Converting the elements of web designs into ex...,112176.0,NaN,Software Engineer; Software Developer; Front E...,Agilent Vee (Domain-Specific Programming Langu...
...,...,...,...,...,...,...,...
55792,Artists and Repertoire Representative,NaN,NaN,NaN,NaN,Front Desk Agent; Front Desk Manager; Receptio...,Computer Literacy; Control Communications; Hea...
55793,Usher/Ticket Taker,NaN,NaN,32949.0,NaN,Sales Manager; Business Development Manager; A...,Cremation; Food Systems Management; Healthcare...
55794,Performer/Teacher,NaN,NaN,45223.0,NaN,Kindergarten Teacher; Substitute Teacher; Elem...,C (Programming Language); Cdna; English Langua...
55795,Salon Owner/Stylist,NaN,NaN,73998.0,NaN,Makeup Artist; Salon Manager; Beauty Consultan...,Attenuation; Control Communications; Good Docu...


In [ ]:
# arr = df.skills.apply(lambda x: x.split("; ")).to_list()
arr = df.skills.apply(lambda x: " ".join(x.split("; "))).to_list()
arr

['Agilent Vee (Domain-Specific Programming Language) Angular (Web Framework) Asp.Net Asp.Net Core Aws Cli (Command Line Interface) Azure Mfa C# (Programming Language) Cloud9 (Software) Codesign Communications Design Patent Drupal Entity Framework Frameworx Javascript (Programming Language) Mapics Ncss (Statistical Software) R.Net Relational Databases Sas Olap Servers',
 'Active Directory Azure Mfa Css Grid Frameworx Javascript (Programming Language) Microsoft Luis Restful Api Sql Services',
 '.Net Framework Agilent Vee (Domain-Specific Programming Language) Angular (Web Framework) Asp.Net Asp.Net Core Asp.Net Mvc 5 Azure Mfa C# (Programming Language) Communications Design Patent Entity Framework Frameworx Html Emails Javascript (Programming Language) Mapics Ncss (Statistical Software) R.Net Sql Services Webapi2',
 'Agilent Vee (Domain-Specific Programming Language) Ajax (Programming Language) Angular (Web Framework) Aws Cli (Command Line Interface) Azure Mfa Bootstrap (Front-End Framew

In [ ]:
# bi_gram = CountVectorizer(ngram_range=(1,2), min_df = 5, binary = True) # 1 means the minimum is unigram, 2 means max is bigrams, 
# bi_gram_vectors = bi_gram.fit_transform(arr)
# bi_gram_vectors

In [ ]:
# pickle.dump(bi_gram, open('bigram.pkl', 'wb'), protocol=4)

In [ ]:
fast_model = FastText(arr,
                      window=3,
                      vector_size=200,
                      alpha=0.1, 
                      min_alpha=0.005)

In [ ]:
pickle.dump(fast_model, open('fastTextModel.pkl', 'wb'), protocol=4)

In [ ]:
fast_arr = df.skills.apply(lambda x: fast_model.wv["".join(x.split("; "))])

In [ ]:
NN = NearestNeighbors(n_neighbors = 5, metric=cosine)
NN.fit(fast_arr.tolist(), df.name)
pickle.dump(NN, open('knn.pkl', 'wb'))

In [14]:
with open("./fastTextModel.pkl", 'rb') as m:
    fastText = pickle.load(m)

with open("./knn.pkl", 'rb') as f:
    model = pickle.load(f)

UnpicklingError: ignored

In [ ]:
# rows = NN.kneighbors(fast_model.wv["Python Data Mining Data Analysis SQL"].reshape(1, -1), return_distance=False)
# for row in rows[0]:
#     instance = df.iloc[row, :]
#     print(instance["name"])
#     print(instance["skills"])

In [11]:
def predict(skillset, fasttext, model, df): 
    x_in = []
    for skill in skillset:
        x_in.append(re.sub(r'\([^)]*\)', '', skill).strip(" "))
        
    x_in = fasttext.wv[" ".join(x_in)].reshape(1, -1)    
    rows = model.kneighbors(x_in, return_distance=False)
    
    res = []
    for row in rows[0]:
        instance = df.iloc[row, :].replace(np.nan, "None").to_dict()
        instance["relatedTitles"] = instance["relatedTitles"].split("; ")
        instance["skills"] = instance["skills"].split("; ")
        res.append(instance)
            
    return res

In [12]:
predict(["Python (Programming Language)", "Data Mining"], fastText, model, df)

NameError: ignored

In [ ]:
df.to_csv("./data/all_titles_jobRec.csv", index=False)

In [ ]:
all_skills = set()
for i in df.skills:
    all_skills = all_skills.union(set([i.lower() for i in i.split(", ")]))
all_skills = sorted(all_skills)

In [ ]:
all_skills

['(american society for quality) asq certified; accounting; analytical skills; analytics; aws cli (command line interface); azure cloud services; cloud security architecture; communications; computer networks; computer sales; customer service; data analysis; data collection; databags; english language; javascript (programming language); linux; python (programming language); scripting',
 '(american society for quality) asq certified; active directory; administrative agencies; analytical skills; aws cli (command line interface); azure cloud services; cloud security architecture; communications; computer literacy; computer sales; customer service; english language; javascript (programming language); leadership; linux; management; microsoft excel; python (programming language); scripting',
 '(american society for quality) asq certified; active directory; agilent vee (domain-specific programming language); aws cli (command line interface); azure cloud services; azure mfa; cloud security arc

## One hot encoding for skills

In [ ]:
# skill2idx = dict(zip(all_skills, range(0, len(all_skills))))
# idx2skill = dict(zip(range(0, len(all_skills)), all_skills))
# api2idx = dict(zip(sorted(api2crawl.keys()), range(0, len(api2crawl))))

In [ ]:
# mapping = defaultdict(list)
# for key in api2crawl:
#     for v in api2crawl[key]:
#         if v in skill2idx:
#             mapping[key].append(skill2idx[v])
# mapping

In [ ]:
# pickle.dump(api2idx, open('mapping.pkl', 'wb'))

In [ ]:
# encoding = []
# for skills in df.skills:
#     curr = np.zeros(len(api2crawl))
#     idx = [api2idx[crawl2api[i.lower()]] for i in skills.split(", ")]
#     curr[idx] = 1
#     encoding.append(curr)
# encoding = np.array(encoding, dtype=int)

In [ ]:
# api2idx

In [ ]:
# encoding.shape

## Building model

In [ ]:
def _validate_vector(u, dtype=None):
    u = np.asarray(u, dtype=int, order='c')
    if u.ndim == 1:
        return u
    raise ValueError("Input vector should be 1-D.")

def distance(u, v):
    u = _validate_vector(u)
    v = _validate_vector(v)
#     nonzero = np.bitwise_or(u != 0, v != 0)
#     unequal_nonzero = np.bitwise_and((u != v), nonzero)
    arr = np.bitwise_and(u, v)
#     a = np.double(unequal_nonzero.sum())
    b = np.double(arr.sum())

    return -b

In [ ]:
df = pd.read_csv("./data/all_titles_jobRec.csv")
with open("./mapping.pkl", 'rb') as m:
    mapping = pickle.load(m)

with open("./knn.pkl", 'rb') as f:
    model = pickle.load(f)

### bigram

In [ ]:
NN = NearestNeighbors(n_neighbors = 5, metric=distance)
NN.fit(bi_gram_vectors.toarray(), df.name)

NearestNeighbors(metric=<function distance at 0x7fdc3a8b2e60>)

In [ ]:
inp = bi_gram.transform(["C (Programming Language); C++ (Programming Language); CI/CD; JavaScript (Programming Language); Node.js"])
rows = NN.kneighbors(inp.toarray(), return_distance=False)
for row in rows[0]:
    instance = df.iloc[row, :]
    print(instance)

In [ ]:
# bigram
def predict(skillset, bigram, model, df): 

    x_in = bi_gram.transform(["; ".join(skillset)])
    rows = model.kneighbors(x_in.toarray(), return_distance=False)
    
    res = []
    for row in rows[0]:
        instance = df.iloc[row, :].replace(np.nan, "None").to_dict()
        instance["relatedTitles"] = instance["relatedTitles"].split("; ")
        instance["skills"] = instance["skills"].split("; ")
        res.append(instance)
            
    return res

In [ ]:
predict(["C (Programming Language)", 
         "C++ (Programming Language)", 
         "CI/CD", 
         "JavaScript (Programming Language)", 
         "Node.js"],
        bi_gram,
        NN,
        df)

[{'name': 'Golang Engineer',
  'description': 'Engineers use mathematical and scientific principles to provide efficient solutions to technical problems. While scientists deliver scientific innovations, engineers are the ones who make these innovations useful for societal and consumer needs.\nOn the most basic level, software engineers write, debug, maintain, and test software that instructs a computer to accomplish certain tasks, such as saving information, performing calculations, etc. A software engineer will convert what needs to happen into one of many programming languages, the most common of which being Java, C++ and Python.. . While many are self taught, most software engineers have obtained an undergraduate or graduate degree in computer science or computer engineering. The career of a software engineer typically follows one of two primary paths: they either progress through greater responsibility and ownership over technical architecture and systems or they progress into mana

In [ ]:
pickle.dump(NN, open('knn.pkl', 'wb'), protocol=4)

In [ ]:
import numpy as np
import pandas as pd
import pickle

### one hot

In [ ]:
# one hot encoding
def predict(skillset, mapping=mapping, model=model, df=df): 

    x_in = np.zeros((1, 2060), dtype=int)
    hash_skills = set()
    for skill in skillset:
        if skill.lower() in mapping:
            hash_skills.add(mapping[skill.lower()])
    hash_skills = list(hash_skills)
    x_in[0, hash_skills] = 1
    
    rows = model.kneighbors(x_in)
    print(rows)
    
    res = []
    for row in rows[1][0]:
        instance = df.iloc[row, :].replace(np.nan, "None").to_dict()
        instance["relatedTitles"] = instance["relatedTitles"].split("; ")
        instance["skills"] = instance["skills"].split("; ")
        res.append(instance)
            
    return res

In [ ]:
%%time
skills = ["Python (Programming Language)", "Data Mining"]
res = predict(skills)
res

(array([[-2., -2., -2., -2., -2.]]), array([[39442, 22053, 45856, 13356, 26071]]))
CPU times: user 574 ms, sys: 270 ms, total: 844 ms
Wall time: 867 ms


[{'name': 'Data Analyst/Developer',
  'description': "A data analyst collects data, organizes and uses it to reach meaningful conclusions. Typically, a data analyst uses computer systems and calculation applications to determine specific factors. They are responsible for digesting the data and creating a report to explain the findings. These resulting reports will help to identify various inefficiencies and other business problems that may exist.\nData analysts research and use data to solve problems, optimize profit, minimize turnover, or streamline logistics, and to help companies make better business decisions. They assess a company’s complex information including sales numbers, market research, or behavior, with the goal of reaching simpler, smarter conclusions, and use their technical expertise to stress test the data for quality and accuracy.. . Data analysts collect, process, and perform statistical analysis of data and translate the numbers into a layman's terms. By identifying

In [ ]:
'data management'.capitalize()

'Data management'